In [ ]:
!nvidia-smi

Mon Apr 17 19:50:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install -q langchain transformers sentence_transformers llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.8/548.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTSimpleVectorIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
import torch
from langchain.llms.base import LLM
from transformers import pipeline

we have created a class and some methods, then called pipeline on the finetuned t5 model


In [ ]:
class customLLM(LLM):
    model_name = "facebook/opt-iml-1.3b"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]

    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())

The model 'OPTForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


transformers are used to extract information and build a q&a bot

In [ ]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

text example

In [ ]:
text1 = """

Othman
PROJECT MANAGER

AREAS OF EXPERTIESE:
IT Management, Project Management, Organizational Restructuring, Business analysis, Risks Analysis, Performance Evaluation, Leadership and Team Building, Problem Resolution, Cost-Efficiency Analysis, Business Process Redesign / Improvement, Scrum Mastery and ERP.

SUMMARY:
•	Intuitive IT management professional offering more than 10 years of experience and well-honed expertise in technology needs analysis and evaluation, assessment of businesses’ core operational procedures, and proficient oversight of top quality enterprise-wide IT projects. Including but not limited to: Budget, Strategy, Business continuity, Business development.
•	Broad project management experience combined with innate ability in identifying innovative procedures to facilitate continuous improvement in business procedures and maintain excellent customer service.
•	Provide leadership and direction for the Information Services Help Desk function, supervised Help Desk Technicians, ensures excellent customer communications, and support of our end users with timely and effective solutions to system’s needs.
•	Knowledgeable and experienced in System Development Life Cycle (SDLC), project management methodologies, requirements definition and gathering, along with outsourcing and off-shoring.

PROFESSIONAL EXPERIENCE:

KO Satellite Inc. | Chantilly, VA | 2007 – Present | IT Manager
•	Oversee, streamline, and ensure overall effectiveness of the IT Department including the implementation of administrative policies and procedures, priorities, and improvement plans
•	Determine IT budget, needs, and recommend cost-efficient IT products to maintain efficiency of business operations and Strategic planning for the department.
•	Responsible for SDLC/development, infrastructure implementation and relocation projects
•	Manage the planning and monitoring of assigned client timelines.
•	Generate and drive project schedules, milestones, and phase review processes.
•	Anticipate and resolve cross-functional project related issues and work with Project Managers to implement improvements.
•	Identify the interdependencies between the different projects in the program and monitor them through the different project lifecycles.
•	Managed projects, teams of Help desk monitoring functions including allocation of personnel completing assignments, training and settings SLA’s, Standards, and day to day operation. Improved customer satisfaction from 65% to 89%
•	Managed team of Developers, Architects, infrastructure, customer support, plus an offshore team of developers
•	Managed SharePoint administration tasks, upgraded SharePoint2007 to 2010 Then to 20113 projects
•	Analyze existing procedures and initiate, as well as implement business process redesign
•	Facilitate improvement in organizational structure, improved efficiency by 30%
•	Maximize personnel efficiency by implementing training strategies
•	Evaluate resource requirements and post-implementation procedures
•	Lead and manage multiple projects concurrently, ensuring projects are within scope, budget and timeline utilizing Agile-Scrum and Waterfall methodologies SAP implementation, Upgrade,  Web, .net projects.
•	Train technical and non-technical personnel on new products and procedures.
•	Evaluation and implementation of new efficient applications used to improve and modernize business processes.
•	Establish and maintain vendor relationships.
•	Create processes and procedures to better serve the IT needs of the company.

Accenture |Chicago, IL | 2006 – 2007 | IT Analyst
•	Project manager of all sized projects with responsibilities including scope definition, requirements gathering, resource scheduling, and project plan maintenance
•	Worked with business stakeholders, direct and manage IT projects from beginning to end including scope definition, project and communication plans, scheduling, QA, training, deployment and post Go-Live support.
•	Present IT Project status updates to senior leadership for support and prioritization
•	Support operations team in portfolio planning and maintenance, include prioritization and resource allocation
•	Support operations team in coordination of software and hardware upgrades across the business
•	Use knowledge and experience of PM methodologies to contribute to organizational Project Management Framework, including customized document templates checklists for 3rd party contractors, and agile processes to help establish and improve project business processes.
•	Refined technical requirements through efficient requirements gathering and project completion
•	Performed and ensured accuracy in technical and functional documentation
•	Maintained operational integrity through SAP business analysis and reviewed, object/ unit testing, and user acceptance testing
•	Managed, motivated, and developed an offshore team

Liberty Tax Service | Chicago, IL | 2005 –2006| PROJECT MANAGER
•	Leading internal and external teamson customer-facing projects
•	Fulfilling contractual commitments within time and budget according to customer requirements
•	Managing all phases of the project (from kickoff to completion) to ensure project success
•	Leading internal and external project teams throughout lifecycle of project
•	Transforming business requirements into specific actions and developing project plans
•	Managing resources and documentation to ensure high quality project deliverables
•	Leading multiple concurrent projects in different project phases
•	Driving issue resolution by engaging team mates and escalating to leadership as appropriate
•	Identifying and implement process improvements to benefit the Program/Project Management team
•	Ensured on time and within budget completion of projects through proficient oversight of a team of technicians
•	Maintained integrity in service by performing technical and business requirements analysis
•	Orchestrated and cost-efficiently completed software implementation projects
•	Managed and maintained coordination in the setup of local infrastructure network and deployment of tax software for the company
•	Systematized plans and oversaw desktop upgrades, security, and maintenance projects
•	Improved existing operations through system recommendations and procurement, maintained project budget

RR Donnelley &Sons Company | Chicago, IL | 2002– 2005| Project Manager
•	Collaborated with business stake holders and customers to ensure proficient business requirements gathering and translation of technical requirements into functional documentations
•	Delegated and ensured balanced distribution of tasks
•	Led, documented, and evaluated change management process on behalf of the business for application changes and updates
•	Designed use cases and test cases for different parts of projects and streamlined quality assurance initiatives
•	Measured project growth and feasibility through project risk analysis and gap analysis, maintained project financials withintime and under budget
•	Managed and maintained coordination of software and hardware deployment projects including SAP ERP projects
•	Managed infrastructure projects including server room, desktop deployment, Microsoft Windows upgrades
•	Interfaced regularly with business executives to define portfolio direction and change priorities
•	Achieved open collaboration with Project Sponsors, Senior Management, Business and IT Stakeholders, PMO to ensure High Profile Client Portfolios are being delivered within Scope, Schedule and Budget.
•	Partnered with other Project Leads and owners in the day-to-day oversight aspects of assigned projects within the portfolio.

RR Donnelley &Sons Company | Chicago, IL | 1999– 2002| Team Lead Technical Support
•	Trained new team members on the products, procedures, and work instructions in a classroom setting.
•	Re-writing the department procedures and SLA’s.
•	Monitoring team member’s calls and rating calls for quality assurance purposes.
•	Team member coaching, providing direction and guidance to successfully service the customers.
•	Spearheaded a team in projects upgrading workstations from Windows 95 to Windows2000
•	Streamlined tasks related to troubleshooting Windows NT, Windows 95, MS Office applications, and LAN administration for Windows 2000
•	Administered and ensured reliability of computer security and applications
•	Managed configuration of TCP / IP and network connectivity
•	Efficiently performed daily troubleshooting, maintenance of user accounts, and daily network backup tape operation

TECHNOLOGY & TOOLS:
C#, .net, ABAP, HTML, XML, VB, JavaScript, and C+, SQL, SQL Query, BODS, HANA,
EDUCATION & TRAINING:
Masters of Information System Management Keller Graduate School of Management
Bachelor Of Science, Computer Engineering, Minor in Computer Science Concordia U. WI
Certified Scrum Master( CSM)
Project Management Professional (PMP) Certification (In Progress)
Project Management, E-commerce, and Web development classes Keller Graduate School of Management Leadership, Organizational Behavior, Time Management, and Team Building.
Visual Basic Courses - Productivity Point Institute, SharePoint, .NET, SAP, ABAP in HR / HCM Functional Module, Functional Configuration HCM, some MM and Order to Cash.
"""

create a document

In [ ]:
from llama_index import Document

text_list = [text1]

documents = [Document(t) for t in text_list]


In [ ]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

getting rid of error messages

In [ ]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [ ]:
response = index.query( "What's this?")
print(response)

Context information is below. 
---------------------
 

Othman 
PROJECT MANAGER

AREAS OF EXPERTIESE:
IT Management, Project Management, Organizational Restructuring, Business analysis, Risks Analysis, Performance Evaluation, Leadership and Team Building, Problem Resolution, Cost-Efficiency Analysis, Business Process Redesign / Improvement, Scrum Mastery and ERP. 

SUMMARY:
•	Intuitive IT management professional offering more than 10 years of experience and well-honed expertise in technology needs analysis and evaluation, assessment of businesses’ core operational procedures, and proficient oversight of top quality enterprise-wide IT projects. Including but not limited to: Budget, Strategy, Business continuity, Business development. 
•	Broad project management experience combined with innate ability in identifying innovative procedures to facilitate continuous improvement in business procedures and maintain excellent customer service. 
•	Provide leadership and direction for the Informa

In [ ]:
response = index.query( "What is the phone number of Abiral")
print(response)

Context information is below. 
---------------------
 

Othman 
PROJECT MANAGER

AREAS OF EXPERTIESE:
IT Management, Project Management, Organizational Restructuring, Business analysis, Risks Analysis, Performance Evaluation, Leadership and Team Building, Problem Resolution, Cost-Efficiency Analysis, Business Process Redesign / Improvement, Scrum Mastery and ERP. 

SUMMARY:
•	Intuitive IT management professional offering more than 10 years of experience and well-honed expertise in technology needs analysis and evaluation, assessment of businesses’ core operational procedures, and proficient oversight of top quality enterprise-wide IT projects. Including but not limited to: Budget, Strategy, Business continuity, Business development. 
•	Broad project management experience combined with innate ability in identifying innovative procedures to facilitate continuous improvement in business procedures and maintain excellent customer service. 
•	Provide leadership and direction for the Informa

gradio UI

In [ ]:

# temp fix for running shell commands on Google Colab

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:0

In [ ]:
import gradio as gr

In [ ]:
index = None

building the model

In [ ]:
def build_the_bot(input_text):
  text_list = [input_text]
  documents = [Document(t) for t in text_list]
  global index
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
  index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
  return('Index saved successfull!!!')

chat, queryleme kısmı

In [ ]:
def chat(chat_history, user_input):

  bot_response = index.query(user_input)
  #print(bot_response)
  response = ""
  for letter in ''.join(bot_response.response): #[bot_response[i:i+1] for i in range(0, len(bot_response), 1)]:
      response += letter + ""
      yield chat_history + [(user_input, response)]


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown('Q&A Bot with Hugging Face Models(flan-t5-large)')
    with gr.Tab("Input Text Document"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Build the Bot!!!")
        text_button.click(build_the_bot, text_input, text_output)
    with gr.Tab("Knowledge Bot"):
#          inputbox = gr.Textbox("Input your text to build a Q&A Bot here.....")
          chatbot = gr.Chatbot()
          message = gr.Textbox ("What is this document about?")
          message.submit(chat, [chatbot, message], chatbot)

demo.queue().launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3e43d40da2b8925789.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
